In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')

#Langchain Tracking and Tracing
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT']=os.getenv('LANGCHAIN_PROJECT')
os.environ['LANGCHAIN_TRACING_V2']="true"

In [4]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model='o1-mini')
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001DBF4174D00> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001DBF6BC0DF0> root_client=<openai.OpenAI object at 0x000001DBF6BB5D30> root_async_client=<openai.AsyncOpenAI object at 0x000001DBF6ABF970> model_name='o1-mini' temperature=1.0 model_kwargs={} openai_api_key=SecretStr('**********')


In [5]:
result = llm.invoke("Capital of Italy")
result.content

'The capital of Italy is **Rome**.'

In [6]:
from langchain_groq import ChatGroq

llm2 = ChatGroq(model='gemma2-9b-it')
print(llm2)

client=<groq.resources.chat.completions.Completions object at 0x000001DBF8232C70> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001DBF8432A90> model_name='gemma2-9b-it' model_kwargs={} groq_api_key=SecretStr('**********')


In [26]:
result = llm2.invoke("Capital of Italy")
result

AIMessage(content='The capital of Italy is **Rome**. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 12, 'total_tokens': 24, 'completion_time': 0.021818182, 'prompt_time': 0.001902316, 'queue_time': 0.156854441, 'total_time': 0.023720498}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--118788bb-0179-417a-8f1f-903690e84d74-0', usage_metadata={'input_tokens': 12, 'output_tokens': 12, 'total_tokens': 24})

In [24]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert AI engineer. Provide answer based on the question"),
        ("user", "{input}")
    ]
)

prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI engineer. Provide answer based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [27]:
chain = prompt | llm2
chain

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI engineer. Provide answer based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021BFC8D4F70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021BFC835FA0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [32]:
response = chain.invoke({"input": "Who is SRK?"})
response.content

'SRK most likely refers to **Shah Rukh Khan**, one of the biggest and most influential movie stars in India, often referred to as the "King of Bollywood". \n\n\nLet me know if you\'d like to know more about him! I can tell you about his:\n\n* **Acting career:** His famous films, awards, and critical acclaim.\n* **Personal life:** His family, philanthropic work, and public image.\n* **Cultural impact:** His influence on Indian cinema and society. \n\nJust ask! 😊 \n\n'

In [34]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [36]:
chain = prompt | llm2 | output_parser

response = chain.invoke({"input": "Who is SRK?"})
response

'SRK most likely refers to **Shah Rukh Khan**, one of the most famous and influential actors in Bollywood.  \n\nHe is widely known as the "King of Bollywood" and has a massive global fan following. \n\nIs there anything else you\'d like to know about Shah Rukh Khan? \n'

In [46]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

output_parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the following {query} in the format {format_instruction}",
    input_variables=["query"],
    partial_variables={"format_instruction": output_parser.get_format_instructions()}
)

In [47]:
chain = prompt | llm2 | output_parser
response = chain.invoke({"query": "Who is SRK?"})
response

{'name': 'Shah Rukh Khan',
 'profession': 'Actor, Producer, Television Personality',
 'nickname': 'SRK',
 'known_for': 'Bollywood films',
 'nationality': 'Indian'}

In [55]:
from langchain_core.output_parsers.xml import XMLOutputParser
from langchain_core.prompts import ChatPromptTemplate

output_parser = XMLOutputParser()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert AI engineer. Provide answer based on the question in XML format"),
        ("user", "{input}")
    ]
)

In [58]:
chain = prompt | llm2
response = chain.invoke({"input": "Who is SRK?"})
response.content

'<response>\n  <answer>SRK refers to Shah Rukh Khan, a renowned Indian actor, producer, and television personality.</answer>\n</response>'

In [4]:
from pydantic import BaseModel, Field
from typing import Annotated

class Product(BaseModel):

    name: Annotated[str, Field(..., description="Name of the prooduct")]
    details: Annotated[str, Field(..., description="Details of the product")]
    price: Annotated[float, Field(..., description="Tentative price of the product in USD")]

In [13]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

output_parser = PydanticOutputParser(pydantic_object=Product)

prompt = PromptTemplate(
    template="Answer the following {query} in the format {format_instruction}",
    input_variables=["query"],
    partial_variables={"format_instruction": output_parser.get_format_instructions()}
)

In [17]:
chain = prompt | llm2 | output_parser
response = chain.invoke({"query": "Rayban meta glass"})
response

Product(name='Ray-Ban Meta Glass', details='Smart glasses with augmented reality features, powered by Ray-Ban and Meta.', price=299.0)